# ENIGMA

## The Enigma Machine

### Rotor Definitions

In [ ]:
rotorDescriptions = { # ring, notch
    'I': ("EKMFLGDQVZNTOWYHXUSPAIBRCJ", 'Q'),
    'II': ("AJDKSIRUXBLHWTMCQGZNPYFVOE", 'E'),
    'III': ("BDFHJLCPRTXVZNYEIWGAKMUSQO", 'V'),
    'IV': ("ESOVPZJAYQUIRHXLNFTGKDCMWB", 'J'),
    'V': ("VZBRGITYUPSDNHLXAWMJQOFECK", 'Z')}

reflectorDescriptions = {
    'UKW B': {'A':'Y','B':'R','C':'U','D':'H','E':'Q','F':'S','G':'L','I':'P','J':'X','K':'N','M':'O','T':'Z','V':'W'},
    'UKW C': {'A':'F','B':'V','C':'P','D':'J','E':'I','G':'O','H':'Y','K':'R','L':'Z','M':'X','N':'W','Q':'T','S':'U'},
}

### Preprocessing

In [ ]:
def num(c):
    return ord(c)-ord('A')

In [ ]:
rotorMaps = {
    name: [num(c) for c in rotor]
    for name,(rotor,notch) in rotorDescriptions.items()
}

rotorInvMaps = dict()
for name, mapping in rotorMaps.items():
    rotorInvMaps[name] = [0] * 26
    for i,c in enumerate(mapping):
        rotorInvMaps[name][c] = i

rotorNotches = {
    name: num(notch)
    for name,(rotor,notch) in rotorDescriptions.items()
}

reflectorMaps = dict()
for name,reflector in reflectorDescriptions.items():
    reflectorMaps[name] = [0] * 26
    for c1,c2 in reflector.items():
        reflectorMaps[name][num(c1)] = num(c2)
        reflectorMaps[name][num(c2)] = num(c1)

### Machine Function

In [ ]:
def EngigmaI(plaintext, wheelOrder, ringSettings, groundSettings, reflectorName, plugConnections):
    # 1. Setup
    # Setup rotors for ease of use
    nRotors = len(wheelOrder)
    notches = [rotorNotches[rotor] for rotor in wheelOrder]
    rotors = [rotorMaps[rotor] for rotor in wheelOrder]
    rotorsInv = [rotorInvMaps[rotor] for rotor in wheelOrder]
    
    reflector = reflectorMaps[reflectorName]

    # Convert settings to numbers for the computer, also implement offsets
    offsets = [num(c) for c in groundSettings]
    offsetSettings = [num(c) for c in ringSettings]
    
    offsets = [(offset - offsetSetting + 26) % 26 for offset,offsetSetting in zip(offsets,offsetSettings)]
    notches = [(notch - offsetSetting + 26) % 26 for notch,offsetSetting in zip(notches,offsetSettings)]
    
    # Create the plugboard map
    plugboard = list(range(26))
    for pair in plugConnections.upper().split(' '):
        plugboard[num(pair[0])] = num(pair[1])
        plugboard[num(pair[1])] = num(pair[0])

    # 2. Perform encryption
    ciphertext = ''
    plaintext = plaintext.upper()  
    for letter in plaintext:
        encryptedLetter = num(letter) 
        if not (0 <= encryptedLetter < 26):
            continue 
        
        # Rotate rotors
        notchActivated = [offsets[iRotor] == notches[iRotor] and iRotor != 0 for iRotor in range(nRotors)]
        for iRotor in range(nRotors-1):
            if notchActivated[iRotor] or notchActivated[iRotor+1]:
                offsets[iRotor] = (offsets[iRotor] + 1) % 26
        offsets[-1] = (offsets[-1] + 1) % 26

        # Begin circuit forward
        encryptedLetter = plugboard[encryptedLetter % 26]
        for rotor,offset in reversed(list(zip(rotors,offsets))):
            encryptedLetter = rotor[(encryptedLetter + offset) % 26] - offset + 26

        encryptedLetter = reflector[encryptedLetter % 26]
        
        # Return backwards
        for rotorInv,offset in zip(rotorsInv,offsets):    
            encryptedLetter = rotorInv[(encryptedLetter + offset) % 26] - offset + 26
        encryptedLetter = plugboard[encryptedLetter % 26]

        # Done
        ciphertext += chr(encryptedLetter + ord('A'))
  
    return ciphertext

## Cracking

We'll want to check our solution candidates for containing english words, so let's get a wordlist.

In [ ]:
import requests
r = requests.get("https://www.mit.edu/~ecprice/wordlist.10000")
#r = requests.get('https://raw.githubusercontent.com/first20hours/google-10000-english/master/google-10000-english.txt')
wordlist = r.text.splitlines()

### Trie Scoring

The enigma machine only encrypts letters: no punctuation including spaces. So we can't split words and check each to see if they've been mapped to a real words.
Instead we can use a trie to run through the the decrypted candidate and check for words efficiently.

Now the problem is there are many very short words, so a solution with many "I"s would score highly, even if the rest was garbage. After some experiementation I found a good solution to was to look for the longest work in the decrypted text. Thus we can make our trie keep track of the length of each word in it, and write our scoring function accordinly to return the length of the longest word found.

In [ ]:
trie = dict()
for word in wordlist:
    curr = trie
    for c in word.upper():
        if c not in curr:
            curr[c] = dict()
        curr = curr[c]
    if 'END' not in curr:
        curr['END'] = len(word)

In [ ]:
def trieMaxScore(text, trie):
    scores = [0]*(1+len(text))
    for i in range(len(text)):
        curr = trie
        j = i
        while j < len(text) and text[j] in curr:
            curr = curr[text[j]]
            j += 1
            if 'END' in curr:
                scores[j] = max(scores[j], curr['END'])
        scores[i+1] = max(scores[i+1], scores[i])
    return scores[-1]

### Our Inputs

In [ ]:
import itertools

Besides the ciphertext, we're given the plugboard connections, the reflector, and hints on the ring settings and ground settings.

In principle this just leaves the 3 of 5 rotors for us to brute force. However the hints I got for the ring and ground settings were dates (both Alan Turing's birthday) so there are multiple date formats to iterate over as well. It's also unclear with hint refers to the ground and ring settings, so that's another place to iterate over.

I did also see some of the other online simulators had different definitions of the orders of ringsettings and ground settings (reversed).

In [ ]:
ciphertext = "YQGGO YFLBR PSWKW JOBFW QRAWY HTLVL DXXMD TSMTJ W"
plugConnections = "IL RQ GE OB KD JF ZW"
reflectorName = "UKW B"

hint1s = [(23,6,12),(6,23,12)]
hint2s = [(23,6,12),(6,23,12)]

# dates forwards or backwards (also order of rings?)
hint1s = hint1s + [tuple(reversed(hint)) for hint in hint1s]
hint2s = hint2s + [tuple(reversed(hint)) for hint in hint2s]

# hints = list(itertools.product(hint1s, hint2s))
hints = list(zip(hint1s,hint2s)) # zip because they are both dates, so their format must match

# hints = hints + [tuple(reversed(hintpair)) for hintpair in hints] # for ring settings vs ground settings

From this we can compute the number of options we have to test. In this case it's very very small.

In [ ]:
import math
print(len(hints) * math.perm(5,3))

### Performing the Crack

In [ ]:
scoreMax = 0
groundSettingsMax = ''
ringSettingsMax = ''
decryptedMax = ''
wheelOrderMax = []

for hint1,hint2 in hints:
    ringSettings = [chr((i-1 +26)%26 + ord('A')) for i in hint1]
    groundSettings = [chr((i-1 +26)%26 + ord('A')) for i in hint2]
    for wheelOrder in itertools.permutations(rotorDescriptions.keys(), 3):
        decrypted = EngigmaI(ciphertext, wheelOrder, ringSettings, groundSettings, reflectorName, plugConnections)
        score = trieMaxScore(decrypted, trie)
        if score > scoreMax:
            scoreMax = score
            groundSettingsMax = groundSettings
            ringSettingsMax = ringSettings
            decryptedMax = decrypted
            wheelOrderMax = wheelOrder

print("Decrypted message:", decryptedMax)

In [ ]:
print("Score of decrypted:", scoreMax)
print("Wheel order:", wheelOrderMax)
print("Ground settings:", groundSettingsMax, list(map(lambda c: num(c)+1,groundSettingsMax)))
print("Ring settings:", ringSettingsMax, list(map(lambda c: num(c)+1,ringSettingsMax)))